In [1]:
# !pip install wandb

In [2]:
# !pip install mediapipe

In [3]:
# !pip install --force-reinstall tensorflow==2.10.1
# !pip install tensorflow-addons
# !pip install tensorflow-datasets

In [13]:
# !pip install tflite-runtime==2.9.1

In [4]:
# !wandb login

In [5]:
!rm -r ./TSSI-POPSIGN

In [6]:
!git clone https://github.com/davidlainesv/TSSI-POPSIGN.git

Cloning into 'TSSI-POPSIGN'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 197 (delta 106), reused 127 (delta 49), pack-reused 0
Receiving objects: 100% (197/197), 1.31 MiB | 10.92 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [23]:
import tensorflow as tf
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow_addons.optimizers import TriangularCyclicalLearningRate
from tensorflow_addons.optimizers import SGDW
from tensorflow.keras.optimizers import SGD
import wandb

import sys
sys.path.insert(1, '/content/TSSI-POPSIGN')

from preprocessing import ResizeIfMoreThan, PadIfLessThan
from datasets.popsign_preprocessing import Preprocessing
from skeleton_graph import tssi_v2
from efficient_net_b0 import EfficientNetB0

NUM_CLASSES = 250

tssi_order = tssi_v2()[1]


def build_sgd_optimizer(initial_learning_rate=0.001,
                        maximal_learning_rate=0.01,
                        step_size=12, momentum=0.0,
                        nesterov=False, weight_decay=1e-7):
    # setup schedule
    learning_rate_schedule = TriangularCyclicalLearningRate(
        initial_learning_rate=initial_learning_rate,
        maximal_learning_rate=maximal_learning_rate,
        step_size=step_size)

    # setup the optimizer
    if weight_decay:
        initial_weight_decay = weight_decay
        maximal_weight_decay = weight_decay * \
            (maximal_learning_rate / initial_learning_rate)
        weight_decay_schedule = TriangularCyclicalLearningRate(
            initial_learning_rate=initial_weight_decay,
            maximal_learning_rate=maximal_weight_decay,
            step_size=step_size)

        optimizer = SGDW(learning_rate=learning_rate_schedule,
                         weight_decay=weight_decay_schedule,
                         momentum=momentum, nesterov=nesterov)
    else:
        optimizer = SGD(learning_rate=learning_rate_schedule,
                        momentum=momentum, nesterov=nesterov)
    return optimizer


def focal_loss(gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002
        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]
        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})
        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.math.add(y_pred, epsilon)
        ce = tf.math.multiply(y_true, -tf.math.log(model_out))
        weight = tf.math.multiply(y_true, tf.math.pow(tf.math.subtract(1., model_out), gamma))
        fl = tf.math.multiply(alpha, tf.math.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

api = wandb.Api()

class_idxs = [45, 246, 211, 131, 63, 119, 46, 140, 11, 184, 149, 105, 56, 50,
              35, 148, 198, 100, 232, 49, 156, 121, 191, 3, 60, 0, 213, 130,
              212, 224, 25, 98, 85, 127, 53, 51, 247, 202, 200, 221, 243, 27,
              173, 196, 136, 21, 194, 22, 2, 26, 218, 81, 39, 170, 28, 165,
              193, 242, 203, 157, 144, 112, 44, 101, 146, 15, 83, 5, 234, 245,
              33, 231, 180, 91, 34, 139, 68, 87, 65, 129, 37, 54, 155, 77, 80,
              103, 8, 116, 227, 36, 120, 215, 93, 47, 189, 163, 228, 43, 249,
              135, 186, 78, 84, 166, 95, 125, 244, 151, 24, 137, 115, 167,
              239, 238, 161, 110, 248, 128, 236, 177, 107, 40, 214, 199, 229,
              69, 205, 74, 159, 154, 240, 207, 108, 145, 7, 162, 58, 160, 99,
              226, 72, 147, 42, 6, 13, 208, 64, 89, 204, 223, 86, 233, 94, 114,
              122, 209, 19, 71, 118, 55, 152, 62, 41, 169, 4, 17, 237, 175,
              133, 92, 219, 178, 66, 172, 183, 188, 111, 153, 117, 220, 12,
              23, 138, 142, 230, 217, 168, 197, 241, 90, 38, 52, 70, 32, 67,
              16, 30, 206, 57, 75, 134, 174, 181, 82, 158, 102, 73, 29, 182,
              113, 76, 179, 185, 195, 106, 59, 14, 97, 225, 132, 9, 216, 187,
              104, 235, 222, 143, 192, 79, 171, 190, 10, 1, 48, 123, 31, 124,
              109, 126, 96, 201, 150, 164, 18, 210, 176, 61, 88, 141, 20]

class ReorderLayer(tf.keras.layers.Layer):
    def __init__(self, class_idxs=[], **kwargs):
        super(ReorderLayer, self).__init__(**kwargs)
        self.class_idxs = tf.constant(class_idxs)

    def call(self, predictions):
      return tf.gather(predictions,
                       indices=self.class_idxs,
                       axis=1)

def build_densenet121_model(dropout=0, optimizer=None, use_focal_loss=False):
    # DOWNLOAD THE WEIGHTS
    # use the latest version of the model
    weights_at = api.artifact('davidlainesv/popsign-testing/run_ejp9qq7p_model:v0')
    # download the directory in which the model is saved
    weights_dir = weights_at.download()
    # print("model: ", weights_dir)

    # ORIGINAL INPUT SHAPE = [frames, 543, 3]
    inputs = Input(shape=[543, 3], name="inputs")

    # EXPAND TO INPUT SHAPE = [1, frames, 543, 3]
    x = tf.expand_dims(inputs, 0)

    # RESHAPE TO INPUT SHAPE = [1, 60, 135, 3]
    x = Preprocessing(tssi_order)(x)
    x = ResizeIfMoreThan(frames=60)(x)
    x = PadIfLessThan(frames=60)(x)
    x = tf.keras.layers.Reshape((60, 135, 3))(x)

    # x = tf.debugging.check_numerics(x)

    # SET CORRECT INPUT SHAPE
    x = DenseNet121(input_shape=[60, 135, 3], weights=None,
                    include_top=False, pooling='avg')(x)
    x = Dropout(dropout)(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)

    # REORDER PREDICTIONS
    predictions = ReorderLayer(class_idxs, name="outputs")(predictions)

    model = Model(inputs=inputs, outputs=predictions)

    # LOAD WEIGHTS
    model.load_weights(weights_dir + "/weights")

    # setup the metrics
    metrics = [
        TopKCategoricalAccuracy(k=1, name='top_1', dtype=tf.float32)
    ]

    # compile the model
    if use_focal_loss:
        loss = focal_loss(alpha=1)
    else:
        loss = "categorical_crossentropy"

    # compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


In [24]:
optimizer = build_sgd_optimizer(initial_learning_rate=0.01, maximal_learning_rate=0.1,
                                momentum=0.9, weight_decay=0, step_size=17724)
model_smart_totem_6 = build_densenet121_model(dropout=0, optimizer=optimizer, use_focal_loss=True)

wandb: Downloading large artifact run_ejp9qq7p_model:v0, 55.57MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.1


In [25]:
# x = tf.constant([[[1, 2]], [[3, 4]]])
# x

In [26]:
# tf.experimental.numpy.full([2, 4, 2], x)

In [27]:
# tf.repeat(x, 4, axis=1)

In [28]:
# tf.fill([1, 3, 2], x)

In [29]:
model_smart_totem_6.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 543, 3)]          0         
                                                                 
 tf.expand_dims_2 (TFOpLambd  (1, None, 543, 3)        0         
 a)                                                              
                                                                 
 preprocessing_2 (Preprocess  (1, None, 135, 3)        0         
 ing)                                                            
                                                                 
 resize_if_more_than_2 (Resi  (1, None, None, 3)       0         
 zeIfMoreThan)                                                   
                                                                 
 pad_if_less_than_2 (PadIfLe  (1, None, None, 3)       0         
 ssThan)                                                   

In [30]:
%timeit model_smart_totem_6.predict(tf.zeros([100, 543, 3]))

4/4 [==============================] - 0s 39ms/step
332 ms ± 69.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Convert model 1

In [31]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model_smart_totem_6)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [32]:
import numpy as np
import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter("./model.tflite")

found_signatures = list(interpreter.get_signature_list().keys())

prediction_fn = interpreter.get_signature_runner("serving_default")
output = prediction_fn(inputs=tf.zeros([100, 543, 3]))
sign = np.argmax(output["outputs"])
print(sign)

158


In [33]:
%timeit prediction_fn(inputs=tf.zeros([100, 543, 3]))

80 ms ± 17.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
